## SI 670 Applied Machine Learning, Week 5:  One-hot encoding, calibration, decision trees, random forests. (Due Thursday October 14 2021, 12:59pm.)

Total: 80 points
Question 1: 30 points
Question 2: 20 points
Question 3: 30 points 

Correct answers and code receive full credit, but partial credit will be awarded if you have the right idea even if your final answers aren't quite right.

Submit your completed notebook file to the Canvas site. IMPORTANT: please name your submitted file si670-hw5-youruniqname.ipynb.

As a reminder, the notebook code you submit must be your own work. Feel free to discuss general approaches to the homework with classmates.


### Put your name here: Cameron Milne

### Put your uniquename here: ccmilne

### Question 1 (30 points)

This question doesn't require coding, in the sense of writing programs that do all the computing of the answer for you: you can figure out the answer by hand and then put your answers into the notebook by defining the correct numpy array in python. Alternatively you could write out your answers in markdown using simple LaTeX tags. (See [here](https://csrgxtu.github.io/2015/03/20/Writing-Mathematic-Fomulars-in-Markdown/) for how to write things with math operators in LaTeX format.)


#### (a) (10 points) One-hot encoding

If you have a dataset with three data points, and each data point has three features. Among them, X2 and X3 are categorical variables:

|    X1	|  X2 	|  X3	|
|----	|----	|----	|
|   1.3	|  a 	| a 	|
|   0.7 |  b 	| c 	|
|   2.1 |  a 	| b     |

Please manually convert this dataset into numerical format with the categorical variables transformed into one-hot encoding. Please keep the order of X1, X2, and X3, and use alphabetical order for the one-hot encoding.

answer:

|  X1	|  X2_a |  X2_b	| X3_a | X3_b | X3_c | 
|----	| ----	|----	|---   |----  | ---- |
|   1.3	|   1 	|  0 	|  1   |  0   | 0
|   0.7 |   0 	|  1 	|  0   |  0   | 1
|   2.1 |   1 	|  0    |  0   |  1   | 0


#### (b) (10 points) Calibration
Recall the calibration curve has the predicted probability as its x-axis and the true probability (also known as the "empirical probability" as its y-axis. Suppose you are given a binary classifier and its predicted probabilities on a test set with 15 data points. The labels of these data points are also given. Please calculate the true probabilities in three bins: \[0, 0.3), \[0.3, 0.7), \[0.7, 1\]. You could further use these probabilities to draw a calibration curve but it's not required for this question. You only need to give the 3 numbers indicating the true probabilities for each bin.  It might help to recall the weather example from class: the "true" empirical probability of rain is just the fraction of times it *actually* rained according to the data, for a given predicted probability (or range of probabilities) from the weatherperson.

|Predicted probability | Label |
|----	               |----   |
|   0.40               |   0   |
|   0.77               |   1   |
|   0.84               |   0   |
|   0.68               |   0   |
|   0.73               |   1   |
|   0.88               |   1   |
|   0.69               |   0   |
|   0.24               |   0   |
|   0.70               |   1   |
|   0.41               |   1   |
|   0.34               |   1   |
|   0.18               |   1   |
|   0.31               |   1   |
|   0.58               |   1   |
|   0.00               |   0   |

answer:

- Bin [0, 0.3): 3 instances, only one rain --> 33%
- Bin [0.3, 0.7): 7 instances, only one rain --> 57%
- Bin [0.7, 1]: 5 instances, only one rain --> 80%

#### (c) (10 points) Random forest parameters

Suppose your current random forest classifier is facing an overfitting situation. Please state whether increasing or decreasing the following parameters can potentially help you reduce overfitting or not, and why?

(i) `n_estimators`

answer: If there are more estimators, a model is less likely to overfit. Increasing the number of trees that are to be built before assessing (taking the maximum voting or averages of predictions) will help to simplify the model and reduce overfitting.

(ii) `max_features`

answer: Max features represents the number of features to consider in looking for the best split. Reducing max features will decrease overfitting.

(iii) `max_depth`

answer: Max depth represents the depth of each tree; the deeper the tree, the more splits and information is captured. Therefore, tree depth is useful to a point, but can hurt testing performance and in this case, reducing max_depth can reduce overfitting. 

(iv) `n_jobs`

answer: The number of jobs (how many processors are to be used in the engine of the model), will have no effect on overfitting. This parameter affects model training speed 


### Question 2 (20 points) Cross-validation for very small datasets.

In our lecture about data leakage, we talked about one simple strategy to help you avoid data leakage: before you do any work with a new dataset, split off a final test dataset, and use this final test dataset as the very last step in your validation. However, when you have a very small dataset, one issue is that it leads to really tiny test sets, which leads to unreliable test evaluation scores. For example, if you have a dataset with a total of 60 samples, and hold back 25% as a final test set, you'll get a final test set with 15 samples. In this case, a single evaluation score based on merely 15 samples could be very unreliable and probably not something to be relied on heavily.

To make evaluation more reliable, we discussed how people usually use *cross-validation* to generate *multiple* evaluation scores, each on a different train/test split of the data. That is, you split the train and test set multiple times and then calculate the average of the resulting test scores. This is the approach we'll use to estimate a more reliable final test set score.  We don't want to use these final test sets to also tune our hyperparameters (to avoid data leakage), so we make sure to learn the model and tune the hyperparameters using only the data in the training split.  To do that, we do a second split *within the training data split* so that we have (i) a training set for the model and (ii) a separate *validation split* that's used to evaluate the model and pick the best setting for the hyperparameters.

Here's the general recipe:

1. Split the whole dataset into $k$ equal folds
2. For $i$ from 1 to $k$   (for each of the $k$ folds)

    a. Take the $i$-th fold as a final test set. 
    
    b. With the remaining data (i.e. combining the other folds), apply a standard train/test split (75%/25%).
    
    c. For each possible tuning value of hyperparameter (in our case, alpha):
    
       - Train the model with the 75% part using a specific hyperparameter choice (for alpha)
             
       - Evaluate the model with the 25% part. This 25% is our "validation set".
             
       Pick the hyperparameter value that gave the best score on the validation set.
            
    d. Once you find the optimal hyperparameter, do the test set predictions on this $i$-th fold and calculate the test score for this $i$-th fold.
  
3. Report the average of the final test set scores you got across all $k$ folds.

To simulate a small dataset scenario, we have provided the code that selects the first 60 samples from the built-in boston dataset.. We've also given you the variable 'alpha_list', which has the range of ridge regression hyperparameter alphe you should use for tuning. 

Write the code that implements the above scheme on this subset of the boston dataset. You can split the whole dataset into k folds (step 1) by using the handy `KFold` function (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html). For each fold, you can then use the default `train_test_split` to get that's fold's training and validation data (step 2b). With this training data, train a Ridge regression model, and use the validation set to evaluate, tuning to find the optimal hyper-parameter alpha value within each fold (step 2c). Get the final test set score using this optimal model (step 2d). Repeat for all $k$ folds to obtain a set of final test scores.

Finally, you need to return the mean value of the $k$ final test scores. This is your final (more reliable) test set prediction.

In [2]:
def answer_two():
    from sklearn.datasets import load_boston
    from sklearn.linear_model import Ridge
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import KFold
    from matplotlib import pyplot as plt
    import operator
    import numpy as np
    
    import warnings
    warnings.filterwarnings("ignore")

    X,y = load_boston(return_X_y=True)
    print(X)
    
    X=X[:60,:]
    y=y[:60]
    alpha_list = [0.001,0.01,0.1,1,10]
    
    print(type(X))
    
    # Your code here
    final_test_scores = []

    K = 5
    kf = KFold(n_splits=K)     

    for train_index, test_index in kf.split(X):
        X_train , X_test = X[train_index], X[test_index]
        y_train , y_test = y[train_index], y[test_index]
        X_train_validate, X_test_validate, y_train_validate, y_test_validate = train_test_split(X_train, y_train, train_size=0.75, random_state=7)

        alphas = {}
        for a in alpha_list:
            model = Ridge(alpha=a).fit(X_train_validate, y_train_validate)
            alphas[str(a)] = model.score(X_test_validate, y_test_validate)
            # print(model.score(X_test_validate, y_test_validate))
            
        top = sorted(alphas.items(), key=operator.itemgetter(1),reverse=True)[:1][0]
        
        new_ridge = Ridge(alpha=top[0]).fit(X_train, y_train)
        score = new_ridge.score(X_test, y_test)

        final_test_scores.append(score)
    
    mean_test_score = sum(final_test_scores) / len(final_test_scores)
    return mean_test_score
    
answer_two()    

[[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 3.9690e+02 4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9690e+02 9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9283e+02 4.0300e+00]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 5.6400e+00]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9345e+02 6.4800e+00]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 7.8800e+00]]
<class 'numpy.ndarray'>


0.009767010370020058

### Question 3 (30 points)  Decision trees.

For this question, we'll work with the Statlog (German Credit Data) dataset that classifies people described by a set of attributes as good or bad credit risks. Download the dataset from [here](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)) (Use the german.data). The last column is the prediction target and the remaining columns are features. 



(a) (20 points) First transform the categorical features into one-hot encodings. Then train a decision tree classifier and a random forest classifier. You should return 6 items as follows: the trained decision tree classifier, the trained random forest classifier, decision tree training accuracy, decision tree test accuracy,  random forest training accuracy, random forest test accuracy. **Please use random_state = 0 for train_test_split, decision tree classifier, and random forest classifier.**

*Hint 1: The columns of categorical features are 0, 2, 3, 5, 6, 8, 9, 11, 13, 14, 16, 18, 19.*

*Hint 2: You may have a problem using `OneHotEncoder` to handle string values and numerical values at the same time. You can transform the string columns first, and then concatenate with the numerical features.*



In [2]:
def answer_three_a():
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    import pandas as pd
    
    german = pd.read_csv('german.data', header=None, sep=' ')
    X = german.iloc[:,:-1]
    y = german.iloc[:,-1]
    
    ct = ColumnTransformer([("OneHote", OneHotEncoder(), [0, 2, 3, 5, 6, 8, 9, 11, 13, 14, 16, 18, 19])], remainder='passthrough')
    transformed = ct.fit_transform(X)
    # print(ct.transformers_) #proof that it's keeping those columns

    ### Decision Tree
    X_train, X_test, y_train, y_test = train_test_split(transformed, y, random_state=0)
    DT_classifier = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)
    train_score_DT = DT_classifier.score(X_train, y_train)
    test_score_DT = DT_classifier.score(X_test, y_test)
    
    ### Random Forest
    X_train, X_test, y_train, y_test = train_test_split(transformed, y, random_state=0)
    RF_classifier = RandomForestClassifier(random_state=0).fit(X_train, y_train)
    train_score_RF = RF_classifier.score(X_train, y_train)
    test_score_RF = RF_classifier.score(X_test, y_test)
    
    return DT_classifier, RF_classifier, train_score_DT, test_score_DT, train_score_RF, test_score_RF
answer_three_a()

(DecisionTreeClassifier(random_state=0),
 RandomForestClassifier(random_state=0),
 1.0,
 0.676,
 1.0,
 0.78)

(b) (10 points) For the Decision Tree Classifier, compute **feature importance** that comes with decision tree classifier in scikit-learn to get the top three most important features. Also do it for the random forest. Are they the same sets of features? Or do you have any interesting findings and comments?




In [3]:
def answer_three_b():
    import numpy as np
    import operator
    DT_classifier, RF_classifier, _,_,_,_ = answer_three_a()
    
    # your code here
    DT = {}
    for i, feat in enumerate(DT_classifier.feature_importances_):
        DT[i] = feat
    DT_important_features = sorted(DT.items(), key=operator.itemgetter(1), reverse=True)[:3]

    RF = {}
    for i, feat in enumerate(RF_classifier.feature_importances_):
        RF[i] = feat    
    RF_important_features = sorted(RF.items(), key=operator.itemgetter(1), reverse=True)[:3]

    return DT_important_features, RF_important_features 
    # DT_important_features is np.array of 3 numbers indicates the indices of top 3 important features in the training features after onehot encoding.
    # DT_important_features is np.array of 3 numbers indicates the indices of top 3 important features in the training features after onehot encoding.

answer_three_b()

([(55, 0.13576646088784447),
  (58, 0.1016517719679399),
  (3, 0.09234069495653098)],
 [(55, 0.09442579022165769),
  (54, 0.07906771820549399),
  (58, 0.07836738053320748)])

Between the Decision Tree Classifier and the Random Forest, two of the same feature indices are within the top three results (55 and 58). Across models, these features are suggestedly important in shaping the outcome.